In [ ]:
import numpy as np
import pandas as pd
import torch
from collections import Counter

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
companyList = pd.read_excel('/content/gdrive/MyDrive/GTMSA/CSE 6242/FinSight Generator Project/resultseval.xlsx')
companyList = list(companyList['ticker'])[:90]

In [106]:
df = pd.read_csv('/content/gdrive/MyDrive/GTMSA/CSE 6242/FinSight Generator Project/News Folder/{}.csv'.format('ida'))
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,Headline,Date,URL,Company
0,"Idacorp boosts dividend by 5.6%, plans to keep...",NaN,https://www.marketwatch.com/story/idacorp-boos...,ida
1,Christian Utility Investor debuts at the top,2012-02-02 10:39:00,https://www.marketwatch.com/story/christian-ut...,ida
2,"Stocks to watch Friday: Google, Williams, Pall",2012-01-20 06:55:00,https://www.marketwatch.com/story/stocks-to-wa...,ida
3,"Updates, advisories and surprises",2005-11-03 20:31:00,https://www.marketwatch.com/story/updates-advi...,ida


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [ ]:
def get_sentiment_scores(headline):
    inputs = tokenizer(headline, return_tensors="pt")
    outputs = finbert(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=1)
    return scores

In [109]:
def evaluator(df):
  table = pd.DataFrame(columns=['ticker', 'positive', 'negative'])
  sentiment_scores = []
  for _, row in df.iterrows():
      headline = row['Headline']
      scores = get_sentiment_scores(headline)
      sentiment_scores.append(scores)
  
  # Convert sentiment scores list to a dataframe
  sentiment_scores_df = pd.DataFrame(torch.cat(sentiment_scores).detach().numpy(), columns=['negative', 'neutral', 'positive'])
  label_list = ['positive', 'negative', 'neutral']
  sentiment_scores_df['label'] = sentiment_scores_df.apply(lambda x: label_list[np.argmax([x.positive,x.negative,x.neutral])], axis = 1)
  label_dict = {'negative': 0, 'neutral': 0, 'positive': 0}
  counter = Counter(sentiment_scores_df['label'].tolist())
  # To keep all the keys in case there is no articles identified as pos/neg
  final_dictionary = {x: label_dict.get(x, 0) + counter.get(x, 0)
                      for x in set(label_dict).union(counter)}
  ticker_info = pd.DataFrame([final_dictionary])
  ticker_info['ticker'] = df['Company']
  ticker_info['total'] = ticker_info.positive +ticker_info.negative+ticker_info.neutral
  ticker_info['net positive percentage'] = (ticker_info.positive/ticker_info.total - ticker_info.negative/ticker_info.total)*100
  ticker_info = ticker_info[['ticker', 'positive', 'negative', 'net positive percentage']]
  return ticker_info

In [113]:
table = pd.DataFrame(columns=['ticker', 'positive', 'negative', 'net positive percentage'])
# Add ticker information and its sentiment distribution to the original dataframe
for ticker in companyList:
  df = pd.read_csv('/content/gdrive/MyDrive/GTMSA/CSE 6242/FinSight Generator Project/News Folder/{}.csv'.format(ticker))
  if df.shape[0] != 0:
    df = df.dropna(subset=['Headline'])
    df = df[df['Headline'] != '']
    table = pd.concat([table, evaluator(df)], axis=0)
  else:
    table1 = pd.DataFrame({'ticker': ticker, 'positive': 0, 'negative': 0, 'net positive percentage': 0.0}, index=[0])
    table = pd.concat([table, table1], axis=0)

In [114]:
table

,ticker,positive,negative,net positive percentage
0,ida,0,2,-50.000000
0,ew,2,4,-22.222222
0,ftnt,3,4,-6.666667
0,anss,2,7,-27.777778
0,anat,0,0,0.000000
...,...,...,...,...
0,fls,2,6,-28.571429
0,dpz,12,6,30.000000
0,rezi,5,9,-22.222222
0,lcii,3,3,0.000000


In [116]:
table.to_csv('newsEval.csv')

In [115]:
companyList == table['ticker']

0    True
0    True
0    True
0    True
0    True
     ... 
0    True
0    True
0    True
0    True
0    True
Name: ticker, Length: 90, dtype: bool